# Transfer Learning CIFAR10

* Train a simple convnet on the CIFAR dataset the first 5 output classes [0..4].
* Freeze convolutional layers and fine-tune dense layers for the last 5 ouput classes [5..9].


In [0]:
# importing the ususal libraries 
import tensorflow as tf 
import pandas as pd 
import numpy as np 
   

### 1. Import CIFAR10 data and create 2 datasets with one dataset having classes from 0 to 4 and other having classes from 5 to 9 

In [0]:
# Loading cifar10 dataset 
from tensorflow.keras.datasets import cifar10


In [0]:
# Splitting the dataset into training and validation sets.
(X_train, y_train), (X_test, y_test) = cifar10.load_data() 

170500096/170498071 [==============================] - 4s 0us/step


In [0]:
print (y_train.shape)
print (X_train.shape)

(50000, 1)
(50000, 32, 32, 3)


In [0]:
# Creating empty lists to further split the dataset from classes 0 to 4 and 5 to 9'
X1_train = []
X1_test = []
X2_train = []
X2_test = []
Y1_train = []
Y1_test = []
Y2_train = []
Y2_test = []

In [0]:
for ix in range(y_train.shape[0]):
    if y_train[ix] < 5:
        # putting training data in set 1(0 to 4 classes) and normalising it
        X1_train.append(X_train[ix]/255.0)
        Y1_train.append(y_train[ix])
    else:
        # putting training data in set 2 (5 to 9) and mormalising it -
        X2_train.append(X_train[ix]/255.0)
        Y2_train.append(y_train[ix])


        # Same for else loop applied to testing data -
for ix in range(y_test.shape[0]):
    if y_test[ix] < 5:
        # put data in set 1
        X1_test.append(X_test[ix]/255.0)
        Y1_test.append(y_test[ix])
    else:
        # put data in set 2
        X2_test.append(X_test[ix]/255.0)
        Y2_test.append(y_test[ix])



In [0]:
# Reshaping the data into an array so that it can be fed to the model.
X1_train = np.asarray(X1_train).reshape((-1, 32, 32, 3)).astype("float32")
X1_test = np.asarray(X1_test).reshape((-1, 32, 32, 3)).astype("float32")
X2_train = np.asarray(X2_train).reshape((-1, 32, 32, 3)).astype("float32")
X2_test = np.asarray(X2_test).reshape((-1, 32, 32, 3)).astype("float32")


In [0]:
print (np.unique(Y1_train))
print (np.unique(Y2_train))

[0 1 2 3 4]
[5 6 7 8 9]


In [0]:
# Renaming the classes from 5 to 9 to 0 to 4 in the 2nd set, so that it can be fed into keras.utils function -
Y2_train=((np.asarray(Y2_train))-5).astype('float32')

In [0]:
Y2_test=((np.asarray(Y2_test))-5).astype('float32')

### 2. Use One-hot encoding to divide y_train and y_test into required no of output classes

In [0]:
# Categorising the classes in set 1 =
Y1_train=tf.keras.utils.to_categorical(Y1_train,num_classes=5)
Y1_test=tf.keras.utils.to_categorical(Y1_test,num_classes=5)

### 3. Build a sequential neural network model which can classify the classes 0 to 4 of CIFAR10 dataset with at least 80% accuracy on test data

In [0]:
# Initializing the seqential model -
model = tf.keras.models.Sequential()


# Adding three convolutional layers with increasing depth , ReLU activation function applied to all Conv layers.
model.add(tf.keras.layers.Conv2D(32,kernel_size=(3,3),activation='relu',input_shape=(32,32,3),name='conv_1'))
model.add(tf.keras.layers.Conv2D(64,kernel_size=(3,3),activation='relu',name='conv_2'))
model.add(tf.keras.layers.Conv2D(128,kernel_size=(3,3),activation='relu',name='conv_3'))

# Adding a dropout layer with 30 % droput threshold -
model.add(tf.keras.layers.Dropout(0.3,name='drop_1'))

# Adding a Pooling layer to reduce inputs to relevant dimensions-
model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2),name='max_1'))

# Flattening the Convolved/Pooled layers to 1D-
model.add(tf.keras.layers.Flatten())

# Adding Fully Connected layers with relU activation functions -
model.add(tf.keras.layers.Dense(150,activation='relu',name='dense_0'))

# Adding first Droput layer to the FC layers with a threshold of 50%.
model.add(tf.keras.layers.Dropout(0.5,name='drop_2'))
model.add(tf.keras.layers.Dense(100,activation='relu',name='dense_1'))

# Adding second Dropout layer with a 40% threshold -
model.add(tf.keras.layers.Dropout(0.4,name='drop_3'))
model.add(tf.keras.layers.Dense(50,activation='relu',name='dense_2'))
model.add(tf.keras.layers.Dense(20,activation='relu',name='dense_3'))

# Adding the output layer with number of classes as output and the activation function as softmax-
model.add(tf.keras.layers.Dense(5,activation='softmax',name='dense_4'))


In [0]:
# Compiling the model with an SGD optimizer with Nestrov momentum optimizing the categorical crossentropy loss function.
optim=(tf.keras.optimizers.SGD(learning_rate=0.01, nesterov=True))
model.compile(optimizer=optim,loss='categorical_crossentropy',metrics=['accuracy'])

In [0]:
# Training and validating the model -
model.fit(X1_train, Y1_train, batch_size=32,epochs=250,validation_data=(X1_test,Y1_test))

Train on 25000 samples, validate on 5000 samples
Epoch 1/250
25000/25000 [==============================] - 7s 262us/sample - loss: 1.5272 - acc: 0.2905 - val_loss: 1.3875 - val_acc: 0.4260
Epoch 2/250
25000/25000 [==============================] - 6s 229us/sample - loss: 1.3309 - acc: 0.4288 - val_loss: 1.2933 - val_acc: 0.4442
Epoch 3/250
25000/25000 [==============================] - 6s 228us/sample - loss: 1.1683 - acc: 0.5142 - val_loss: 1.0537 - val_acc: 0.5868
Epoch 4/250
25000/25000 [==============================] - 6s 231us/sample - loss: 1.0773 - acc: 0.5528 - val_loss: 1.0694 - val_acc: 0.5616
Epoch 5/250
25000/25000 [==============================] - 6s 230us/sample - loss: 1.0176 - acc: 0.5857 - val_loss: 0.9526 - val_acc: 0.6132
Epoch 6/250
25000/25000 [==============================] - 6s 234us/sample - loss: 0.9707 - acc: 0.6098 - val_loss: 0.9174 - val_acc: 0.6362
Epoch 7/250
25000/25000 [==============================] - 6s 232us/sample - loss: 0.9245 - acc: 0.6292 -

In [0]:
# Evaluating the model -
score = model.evaluate(X1_test, Y1_test)
print('Test loss:', score[0])
print('Test accuracy:', score[1])    

5000/5000 [==============================] - 1s 108us/sample - loss: 1.2712 - acc: 0.8046
Test loss: 1.2711625506401063
Test accuracy: 0.8046


### 4. In the model which was built above (for classification of classes 0-4 in CIFAR10), make only the dense layers to be trainable and conv layers to be non-trainable

In [0]:
# Applying transfer learning, here both Domain and Tasks are similar, hence beneficial to just train the Dense Layers, as Data is also abundant-
# Freezing all layers except the FC layers -
for layer in model.layers:
  if('dense' not in layer.name): 
    layer.trainable = False

### 5. Utilize the the model trained on CIFAR 10 (classes 0 to 4) to classify the classes 5 to 9 of CIFAR 10  (Use Transfer Learning) <br>
Achieve an accuracy of more than 85% on test data

In [0]:
# Categorizing the class variables of set 2 -
Y2_train=tf.keras.utils.to_categorical(Y2_train,num_classes=5)
Y2_test=tf.keras.utils.to_categorical(Y2_test,num_classes=5)

In [0]:
# Training the model on  Dataset 2 -
model.fit(X2_train, Y2_train, batch_size=32,epochs=50,validation_data=(X2_test,Y2_test))

Train on 25000 samples, validate on 5000 samples
Epoch 1/50
25000/25000 [==============================] - 6s 236us/sample - loss: 0.2632 - acc: 0.9102 - val_loss: 0.5112 - val_acc: 0.8352
Epoch 2/50
25000/25000 [==============================] - 6s 231us/sample - loss: 0.2574 - acc: 0.9126 - val_loss: 0.4053 - val_acc: 0.8634
Epoch 3/50
25000/25000 [==============================] - 6s 231us/sample - loss: 0.2367 - acc: 0.9208 - val_loss: 0.4431 - val_acc: 0.8496
Epoch 4/50
25000/25000 [==============================] - 6s 230us/sample - loss: 0.2298 - acc: 0.9230 - val_loss: 0.4897 - val_acc: 0.8298
Epoch 5/50
25000/25000 [==============================] - 6s 230us/sample - loss: 0.2168 - acc: 0.9271 - val_loss: 0.4138 - val_acc: 0.8598
Epoch 6/50
25000/25000 [==============================] - 6s 232us/sample - loss: 0.2081 - acc: 0.9301 - val_loss: 0.5055 - val_acc: 0.8396
Epoch 7/50
25000/25000 [==============================] - 6s 231us/sample - loss: 0.2009 - acc: 0.9315 - val_lo

In [0]:
# Evaluating the Model on set 2 -
score = model.evaluate(X2_test, Y2_test)
print('Test loss:', score[0])
print('Test accuracy:', score[1])    

5000/5000 [==============================] - 1s 109us/sample - loss: 0.5386 - acc: 0.8768
Test loss: 0.5385919322371483
Test accuracy: 0.8768


# Text classification using TF-IDF

### 6. Load the dataset from sklearn.datasets

In [0]:
# Importing the relevant dataset- 
from sklearn.datasets import fetch_20newsgroups

In [0]:
import numpy as np

In [0]:
# Choosing a subset of the categories -
categories = ['alt.atheism', 'soc.religion.christian', 'comp.graphics', 'sci.med']

### 7. Training data

In [0]:
# Fetching the trainig data-
twenty_train = fetch_20newsgroups(subset='train', categories=categories, shuffle=True, random_state=42)

### 8. Test data

In [0]:
# Fetching the test data -
twenty_test = fetch_20newsgroups(subset='test', categories=categories, shuffle=True, random_state=42)

###  a.  You can access the values for the target variable using .target attribute 
###  b. You can access the name of the class in the target variable with .target_names


In [0]:
# Eyeballing the training and testing data -

In [0]:
twenty_train.target.shape

(2257,)

In [0]:
y_train=twenty_train.target

In [0]:
y_test=twenty_test.target

In [0]:
twenty_test.target.shape

(1502,)

In [0]:
twenty_test.target_names

['alt.atheism', 'comp.graphics', 'sci.med', 'soc.religion.christian']

### 9.  Now with dependent and independent data available for both train and test datasets, using TfidfVectorizer fit and transform the training data and test data and get the tfidf features for both

In [0]:
# Importing the TfidfVectorizer -
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer()
                                 
                                


In [0]:
# Fitting and transforming the training and testing data into Tfidf matrices -
X_train=tfidf_vectorizer.fit_transform(twenty_train.data)
X_test=tfidf_vectorizer.fit_transform(twenty_test.data)

In [0]:
X_train.shape

(2257, 35788)

In [0]:
y_train.shape

(2257,)

In [0]:
X_test.shape

(1502, 30786)

In [0]:
y_test.shape

(1502,)

### 10. Use logisticRegression with tfidf features as input and targets as output and train the model and report the train and test accuracy score

In [0]:
# Importing Logistic regression model from scikit learn -
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()

# Fitting and trainig the model -
model.fit(X_train,y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [0]:
# Printing the accuracy of teh model on the training data -
model.score(X_train,y_train)

0.9827204253433761

In [0]:
# Fitting the model on testing data -
model.fit(X_test,y_test)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [0]:
# Accuracy of the model on the testing data -
model.score(X_test,y_test)

0.9853528628495339